# Environment Setup and Data Loading

In [12]:
import scanpy as sc
import numpy as np
import pandas as pd
import os
import sys
import anndata as ad
import random
import functions_degs
import importlib

PROJECT_DIR = "D:/Github/SRF_Linda_RNA"
WORKING_DIR = os.path.join(PROJECT_DIR, "combine_data")
os.chdir(WORKING_DIR)
sys.path.insert(0, WORKING_DIR)

importlib.reload(functions_degs)

# Set seeds for all random number generators
random_seed = 0
np.random.seed(random_seed)
random.seed(random_seed)

In [13]:
# Set up directories

# REMOVE_DOUBLETS = True
REMOVE_DOUBLETS = False

FIX_TRESHOLD = True
# FIX_TRESHOLD = False

if FIX_TRESHOLD:
    BASE_RESULTS_DIR = os.path.join(WORKING_DIR, "results_from_raw")
else:
    if REMOVE_DOUBLETS:
        BASE_RESULTS_DIR = os.path.join(WORKING_DIR, "results_from_raw_percentile_threshold", "doublets_removed")
    else:
        BASE_RESULTS_DIR = os.path.join(WORKING_DIR, "results_from_raw_percentile_threshold")

INPUT_DIR = BASE_RESULTS_DIR
adata_path = os.path.join(INPUT_DIR, 'annotation_final.h5ad')

PARENT_OUTPUT_DIR = os.path.join(INPUT_DIR, "DEGs_mapmycells_L2")

# CUSTOM_ANALYSIS =  None
CUSTOM_ANALYSIS =  "FC_0_25"

DEG_BY = "mapmycells_second_layer"

if CUSTOM_ANALYSIS is not None:
    PARENT_OUTPUT_DIR = PARENT_OUTPUT_DIR + CUSTOM_ANALYSIS

PLOT_OUTPUT_DIR = os.path.join(PARENT_OUTPUT_DIR, 'plots')
DGE_OUTPUT_DIR = os.path.join(PARENT_OUTPUT_DIR, 'dge_res')
BIOMARKER_OUTPUT_DIR = os.path.join(PARENT_OUTPUT_DIR, 'biomarkers')

# Create output directories if they don't exist
os.makedirs(PLOT_OUTPUT_DIR, exist_ok=True)
os.makedirs(DGE_OUTPUT_DIR, exist_ok=True)
os.makedirs(BIOMARKER_OUTPUT_DIR, exist_ok=True)

print(f"Input directory: {INPUT_DIR}")
print(f"Plot output directory (Overall/Genotype): {PLOT_OUTPUT_DIR}")
print(f"DGE output directory (Overall/Genotype): {DGE_OUTPUT_DIR}")
print(f"Biomarker output directory: {BIOMARKER_OUTPUT_DIR}")

# Configure scanpy settings
sc.settings.figdir = PLOT_OUTPUT_DIR
sc.settings.set_figure_params(dpi=150, facecolor='white')

Input directory: D:/Github/SRF_Linda_RNA\combine_data\results_from_raw
Plot output directory (Overall/Genotype): D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\plots
DGE output directory (Overall/Genotype): D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\dge_res
Biomarker output directory: D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\biomarkers


In [14]:
# Load data
print(f"\nLoading dataset from {adata_path}")
adata = sc.read_h5ad(adata_path)


Loading dataset from D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\annotation_final.h5ad


# Data Preprocessing

In [15]:
# Save current settings
original_max_rows = pd.get_option('display.max_rows')

# Set to None to display all rows
pd.set_option('display.max_rows', None)

# Check original clusters:
print(f"Original {DEG_BY} clusters:")
print(adata.obs[DEG_BY].value_counts())

# Reset to original settings
pd.set_option('display.max_rows', original_max_rows)

Original mapmycells_second_layer clusters:
mapmycells_second_layer
DG Glut                               3724
CA1-ProS Glut                         3379
Oligo NN                              1828
CA3 Glut                              1546
L2/3 IT ENT Glut                      1231
L2/3 IT PPP Glut                      1041
Astro-TE NN                           1005
L6 CT CTX Glut                         991
Microglia NN                           931
L2/3 IT CTX Glut                       848
MEA Slc17a7 Glut                       823
OPC NN                                 822
Sst Gaba                               706
L2/3 IT PIR-ENTl Glut                  696
SUB-ProS Glut                          658
LA-BLA-BMA-PA Glut                     640
L6b/CT ENT Glut                        545
L4/5 IT CTX Glut                       498
Pvalb Gaba                             427
L2 IT ENT-po Glut                      316
Vip Gaba                               315
L5/6 IT TPE-ENT Glut          

In [16]:
# Check adata structure
print("\nAnnData object summary:")
print(adata)
print("\nAvailable layers:", list(adata.layers.keys()))
print("Raw data available:", adata.raw is not None)
if adata.raw:
    print("Raw data shape:", adata.raw.X.shape)


AnnData object summary:
AnnData object with n_obs × n_vars = 28026 × 26870
    obs: 'sample', 'condition', 'genotype', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden_0.4', 'ISO_majority_voting', 'ISO_conf_score', 'DG_majority_voting', 'DG_conf_score', 'mapmycells_first_layer', 'mapmycells_second_layer', 'cell_type', 'highlight'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'DG_majority_voting_colors', 'ISO_majority_voting_colors', 'cell_type_colors', 'condition_colors', 'genotype_colors', 'highlight_colors', 'hvg', 'leiden_0.4', 'leiden_0.4_colors', 'log1p', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

Available layers: []
Raw data available: True
Raw data shape: (28026, 33685)


In [17]:
# Create the 'for_DEGs' layer: Normalized and log1p transformed counts
# Assuming raw counts are in adata.raw.X
print("\nCreating 'for_DEGs' layer...")
if adata.raw is not None and adata.raw.X is not None:
    # Create a temporary AnnData with raw counts to perform normalization and log1p
    adata_for_dge = ad.AnnData(adata.raw.X.copy()) # type: ignore
    adata_for_dge.obs_names = adata.obs_names # type: ignore
    adata_for_dge.var_names = adata.raw.var_names # type: ignore

    print("Normalizing total counts (target_sum=1e4)...")
    sc.pp.normalize_total(adata_for_dge, target_sum=1e4)

    print("Applying log1p transformation...")
    sc.pp.log1p(adata_for_dge)

    # Ensure the gene order matches the main adata object
    adata_for_dge = adata_for_dge[:, adata.var_names].copy() # type: ignore

    print("Storing result in adata.layers['for_DEGs']...")
    adata.layers['for_DEGs'] = adata_for_dge.X.copy() # type: ignore
    print("'for_DEGs' layer created with shape:", adata.layers['for_DEGs'].shape)
else:
    print("Warning: adata.raw.X not found. Cannot create 'for_DEGs' layer from raw counts.")
    print("DGE analysis will proceed using adata.X, which might be scaled.")


Creating 'for_DEGs' layer...
Normalizing total counts (target_sum=1e4)...
Applying log1p transformation...
Storing result in adata.layers['for_DEGs']...
'for_DEGs' layer created with shape: (28026, 26870)


In [18]:
print("\nMetadata check:")
print("condition:", list(adata.obs.condition.unique()))
print("genotype:", list(adata.obs.genotype.unique()))
print(f"{DEG_BY}:", list(adata.obs[DEG_BY].unique()))


Metadata check:
condition: ['Control', 'Mutant']
genotype: ['Emx1', 'Nestin']
mapmycells_second_layer: ['Astro-TE NN', 'L6 IT CTX Glut', 'CA3 Glut', 'Monocytes NN', 'L5 ET CTX Glut', 'L2/3 IT PPP Glut', 'CA1-ProS Glut', 'MEA Slc17a7 Glut', 'Astro-OLF NN', 'COAp Grxcr2 Glut', 'LA-BLA-BMA-PA Glut', 'DG Glut', 'Oligo NN', 'NTS Phox2b Glut', 'Sst Gaba', 'L2/3 IT PIR-ENTl Glut', 'SUB-ProS Glut', 'Lymphoid NN', 'L6b CTX Glut', 'L5 NP CTX Glut', 'L2/3 IT CTX Glut', 'L2/3 IT ENT Glut', 'HPF CR Glut', 'Microglia NN', 'Vip Gaba', 'L6b/CT ENT Glut', 'L2 IT ENT-po Glut', 'OB-in Frmd7 Gaba', 'CA2-FC-IG Glut', 'L6 CT CTX Glut', 'RHP-COA Ndnf Gaba', 'SC Lef1 Otx2 Gaba', 'DG-PIR Ex IMN', 'OPC NN', 'Sncg Gaba', 'L4/5 IT CTX Glut', 'IA Mgp Gaba', 'LSX Prdm12 Slit2 Gaba', 'IT AON-TT-DP Glut', 'Lamp5 Lhx6 Gaba', 'CEA-BST Ebf1 Pdyn Gaba', 'Lamp5 Gaba', 'IT EP-CLA Glut', 'Pvalb Gaba', 'VLMC NN', 'L6b EPd Glut', 'OB Meis2 Thsd7b Gaba', 'L5/6 IT TPE-ENT Glut', 'IC Six3 En2 Gaba', 'ENTmv-PA-COAp Glut', 'LDT V

# Differential Gene Expression Analysis

In [19]:
# Overall DGE (Mutant vs Control) using the 'for_DEGs' layer, grouped by the DEG_BY
print(f"\nRunning Overall DGE (grouped by {DEG_BY})...")
dge_results = functions_degs.run_overall_dge(
    adata,
    grouping_key=DEG_BY,
    dge_output_dir=DGE_OUTPUT_DIR,
    plot_output_dir=PLOT_OUTPUT_DIR,
    layer='for_DEGs' if 'for_DEGs' in adata.layers else None
)


Running Overall DGE (grouped by mapmycells_second_layer)...

Starting both genomes Differential Gene Expression analysis (Mutant vs Control)...
  Running DGE for group: Astro-TE NN (using key 'mapmycells_second_layer')
    Initial genes: 26870
    Filtering genes: Requiring expression in >= 101 cells (max of 3 absolute and 10.0% of 1005)...
    Genes after filtering: 5798
      Calculating mean expression for 5798 genes...
      Using data from layer: for_DEGs
Added mean expression columns.
    DGE results saved to D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\dge_res\both_geno_cond_comp\dge_both_genomes_Astro-TE_NN_mut_vs_ctrl.csv
    DGE results saved to D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\dge_res\both_geno_cond_comp\dge_both_genomes_Astro-TE_NN_mut_vs_ctrl.xlsx
    DEBUG: Attempting to create list_output_dir: 'D:\Github\SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\dge_res\both_gen

In [20]:
# Genotype-Specific DGE (Mutant vs Control within each genotype) using the 'for_DEGs' layer, grouped by the DEG_BY
print(f"\nRunning Genotype-Specific DGE (grouped by {DEG_BY})...")
dge_by_genotype = functions_degs.run_genotype_specific_dge(
    adata,
    grouping_key=DEG_BY,
    dge_output_dir=DGE_OUTPUT_DIR,
    plot_output_dir=PLOT_OUTPUT_DIR,
    layer='for_DEGs' if 'for_DEGs' in adata.layers else None
)


Running Genotype-Specific DGE (grouped by mapmycells_second_layer)...

Starting Genotype-Specific DGE analysis...
Created 'genotype_condition' column with categories: ['Emx1_Control', 'Emx1_Mutant', 'Nestin_Control', 'Nestin_Mutant']

Processing group: Astro-TE NN (using key 'mapmycells_second_layer')
  Running DGE for Emx1 genotype...
    Filtering genes for Emx1 (Initial: 26870)
    Filtering genes: Requiring expression in >= 38 cells (max of 3 absolute and 10.0% of 372)...
    Genes after filtering: 5030
      Calculating mean expression for 5030 genes (Emx1, group Astro-TE NN)...
      Using data from layer: for_DEGs
      Added mean expression columns (Emx1, group Astro-TE NN).
    Finished DGE for Emx1, group Astro-TE NN. Found 5030 ranked genes.
    DGE results saved to D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\dge_res\geno_spec_cond_comp\dge_Emx1_Astro-TE_NN_mut_vs_ctrl.csv
    DGE results saved to D:/Github/SRF_Linda_RNA\combine_data\resu

In [21]:
# Genotype Comparison DGE (Nestin vs Emx1 within each condition) using the 'for_DEGs' layer, grouped by the DEG_BY
print(f"\nRunning Genotype Comparison DGE (grouped by {DEG_BY})...")
dge_genotype_within_condition = functions_degs.run_genotype_comparison_dge(
    adata,
    grouping_key=DEG_BY,
    dge_output_dir=DGE_OUTPUT_DIR,
    plot_output_dir=PLOT_OUTPUT_DIR,
    layer='for_DEGs' if 'for_DEGs' in adata.layers else None
)


Running Genotype Comparison DGE (grouped by mapmycells_second_layer)...

Starting DGE analysis: Genotype comparison within conditions...

Processing group: Astro-TE NN (using key 'mapmycells_second_layer')
  Running DGE for Control condition (Nestin vs Emx1)...
    Initial genes for Control condition: 26870
    Filtering genes for Control: Requiring expression in >= 63 cells (max of 3 absolute and 10.0% of 626)...
    Genes after filtering for Control: 6256
      Calculating mean expression for 6256 genes (Control condition, group Astro-TE NN)...
      Using data from layer: for_DEGs
      Added mean expression columns (Control condition, group Astro-TE NN).
    Finished DGE for Control, group Astro-TE NN. Found 6256 ranked genes.
    DGE results saved to D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\dge_res\cond_spec_geno_comp\dge_Control_cond_Astro-TE_NN_Nestin_vs_Emx1.csv
    DGE results saved to D:/Github/SRF_Linda_RNA\combine_data\results_from_ra

In [22]:
# Cell Type Comparison DGE (Each cell type vs Rest) using the 'for_DEGs' layer, using the DEG_BY
print(f"\nRunning Cell Type Comparison DGE (Markers for {DEG_BY})...")
dge_mapmycells_second_layer_markers = functions_degs.run_cluster_comparison_dge(
    adata,
    grouping_key=DEG_BY, 
    dge_output_dir=BIOMARKER_OUTPUT_DIR, 
    plot_output_dir=BIOMARKER_OUTPUT_DIR,
    layer='for_DEGs' if 'for_DEGs' in adata.layers else None,
    method='wilcoxon' # or 't-test'
)


Running Cell Type Comparison DGE (Markers for mapmycells_second_layer)...

Starting Cluster Comparison DGE analysis (Markers for 'mapmycells_second_layer') into D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\biomarkers...
  Initial genes: 26870
  Filtering genes globally: Requiring expression in >= 3 cells...
  Genes after global filtering: 26870
  Proceeding with DGE for 137 groups. Removed groups: ['LDT Vsx2 Nkx6-1 Nfib Glut', 'NI-RPO Gata3 Nr4a2 Gaba', 'RT-ZI Gnb3 Gaba', 'SPVC Mafa Glut', 'SNc-VTA-RAmb Foxa1 Dopa', 'CS-RPO Meis2 Gaba', 'AHN-SBPV-PVHd Pdrm12 Gaba', 'NLOT Rho Glut', 'PAG-ND-PCG Onecut1 Gaba', 'MB-MY Tph2 Glut-Sero', 'SCsg Pde5a Glut', 'MEA-BST Lhx6 Sp9 Gaba', 'Bergmann NN', 'LSX Sall3 Pax6 Gaba', 'MG-POL-SGN Nts Glut', 'LGv-ZI Otx2 Gaba', 'LDT-PCG St18 Gaba', 'SCH Six6 Cdc14a Gaba', 'PRP Otp Gly-Gaba', 'DMH Hmx2 Gaba', 'STN-PSTN Pitx2 Glut', 'OT D3 Folh1 Gaba', 'MRN-PPN-CUN Pax8 Gaba', 'SCs Lef1 Gli3 Gaba', 'PDTg-PCG Pax6 Gaba', 'PH-a

d:\Programs\miniforge3\envs\bioinf\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:456: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "names"] = self.var_names[global_indices]
d:\Programs\miniforge3\envs\bioinf\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:458: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "scores"] = scores[global_indices]
d:\Programs\miniforge3\envs\bioinf\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:461: PerformanceWarning: DataFrame is highly 

  rank_genes_groups finished.
    Calculating mean expression for Cluster ABC NN vs Rest...
      Using data from layer: for_DEGs
      Added mean expression columns for Cluster ABC NN.
    Top 50 Cluster marker results saved to D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\biomarkers\dge_cluster_comparison_ABC_NN_vs_Rest_top50.csv
    Top 50 Cluster marker results saved to D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\biomarkers\dge_cluster_comparison_ABC_NN_vs_Rest_top50.xlsx
    DEBUG: Attempting to create list_output_dir: 'D:\Github\SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\biomarkers\sig_deg_lists\ABC_NN'
    DEBUG: Successfully created/confirmed directory: 'D:\Github\SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\biomarkers\sig_deg_lists\ABC_NN'
    DEBUG: Directory 'D:\Github\SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\biomarkers\sig_de

In [23]:
print("\nAll analysis and output generation complete.")
print(f"Overall/Genotype DGE outputs saved in: {DGE_OUTPUT_DIR}")
print(f"Biomarker DGE outputs saved in: {BIOMARKER_OUTPUT_DIR}")


All analysis and output generation complete.
Overall/Genotype DGE outputs saved in: D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\dge_res
Biomarker DGE outputs saved in: D:/Github/SRF_Linda_RNA\combine_data\results_from_raw\DEGs_mapmycells_L2FC_0_25\biomarkers
